In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tqdm import tqdm
from tensorflow.keras.models import load_model, Model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input

from mtcnn.mtcnn import MTCNN
# Path to celeb dataset
PATH = '../data/img_celeba_preprocess'
# celeb_data = pd.read_csv('identity_CelebA.txt', sep=" ", header=None)
# celeb_data.columns = ["image", "label"]
# train_df = pd.read_csv('train.csv')
# valid_df = pd.read_csv('valid.csv')

# train_df = pd.read_csv('./labels/train.pre.small.csv')
# valid_df = pd.read_csv('./labels/valid.pre.small.csv')
train_df = pd.read_csv('./labels/train.pre.csv')
valid_df = pd.read_csv('./labels/valid.pre.csv')

Using TensorFlow backend.


In [2]:
import tensorflow as tf
# dir(tf.keras.applications)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    except RuntimeError as e:
        # 프로그램 시작시에 가장 장치가 설정되어야만 합니다
        print(e)

#### Visualize few images of the same person

In [4]:
detector = MTCNN()
# model = load_model('outputs/siamese.densenet169.v5.h5', compile=False)
model = load_model('outputs/siamese.densenet169.all.v5.h5', compile=False)

inp = model.input[0]
base_model = model.layers[3]
infer_model = Model(inp, base_model(inp))
infer_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
anchor (InputLayer)          [(None, 224, 224, 3)]     0         
_________________________________________________________________
model (Model)                (None, 128)               13561024  
Total params: 13,561,024
Trainable params: 13,402,624
Non-trainable params: 158,400
_________________________________________________________________


In [6]:
# Load pre-trained model
# model = load_model('siamese_xception.model.h5', compile=False)
import cv2
FACE_DEFAULT_SHAPE = (224, 224)

preds = np.zeros( (len(valid_df), 128))
for k, file in tqdm(enumerate(valid_df['image'])):
    img = image.load_img(os.path.join(PATH, file))
    img = image.img_to_array(img)
    # new_img = preprocess(detector, img, FACE_DEFAULT_SHAPE)
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)
    preds[k] = infer_model.predict(x)[0]

# np.save('outputs/valid_vec.128', preds)
np.save('outputs/valid_vec.all.128', preds)

6404it [06:18, 16.94it/s]


In [7]:
# Load pre-trained model
# model = load_model('siamese_xception.model.h5', compile=False)
import cv2
FACE_DEFAULT_SHAPE = (224, 224)

preds = np.zeros( (len(train_df), 128))
for k, file in tqdm(enumerate(train_df['image'])):
    img = image.load_img(os.path.join(PATH, file))
    img = image.img_to_array(img)
    # new_img = preprocess(detector, img, FACE_DEFAULT_SHAPE)
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)
    preds[k] = infer_model.predict(x)[0]

# np.save('outputs/train_vec.128', preds)
np.save('outputs/train_vec.all.128', preds)

51179it [49:36, 17.20it/s]
